In [ ]:
#Текста, имена
#! git clone https://github.com/NVIDIA/apex
! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ../input/tinkoffapex/apex-master
#! cd apex
#! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/tinkoffapex/apex-master"))
#print(os.listdir("../input/glove-global-vectors-for-word-representation"))
#print(os.listdir("../input/jigsaw-unintended-bias-in-toxicity-classification"))
#print(os.listdir("../input/fasttext-crawl-300d-2m"))

# Any results you write to the current directory are saved as output.

In [ ]:
#!pip install apex

In [ ]:
# Installing Nvidia Apex

#!git clone https://github.com/NVIDIA/apex.git && cd apex && python setup.py install --cuda_ext --cpp_ext

In [ ]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import pkg_resources
import seaborn as sns
import time
import scipy.stats as stats
import gc
import re
import operator 
import sys
from sklearn import metrics
from sklearn import model_selection
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from nltk.stem import PorterStemmer
from sklearn.metrics import roc_auc_score
%load_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
from apex import amp
import shutil

In [ ]:

device=torch.device('cuda')

In [ ]:
#1804874
#MAX_SEQUENCE_LENGTH = 256
MAX_SEQUENCE_LENGTH =256
SEED = 1234
EPOCHS = 3
Data_dir="../input/datatinkoff/"
Input_dir = "../input"
WORK_DIR = "../working/" 
#num_to_load= 1600000                         #Train size to match time limit
#valid_size= 204874                         #Validation Size


#d = 5000
#num_to_load = 4000
#valid_size = 1000
d = 473141
num_to_load= 373141  #

valid_size= 100000




#valid_size= 5000



#num_to_load= 20000                       #Train size to match time limit
#valid_size= 10000
TOXICITY_COLUMN = 'target'

In [ ]:
# Add the Bart Pytorch repo to the PATH
# using files from: https://github.com/huggingface/pytorch-pretrained-BERT


package_dir_a = "../input/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT"
sys.path.insert(0, package_dir_a)

from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam, OpenAIAdam
print(os.listdir("../working"))

In [ ]:
# Translate model from tensorflow to pytorch
#BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'
BERT_MODEL_PATH   = "../input/russian-bert-pavlov/"
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
     BERT_MODEL_PATH + 'bert_model.ckpt',
BERT_MODEL_PATH + 'bert_config.json',
WORK_DIR + 'pytorch_model.bin')
#shutil.copyfile("../input/weightbert/1_bert_pytorch.bin", WORK_DIR + 'pytorch_model.bin')
#shutil.copyfile(BERT_MODEL_PATH + 'bert_pytorch.bin', WORK_DIR + 'bert_pytorch.bin')
#comment
#shutil.copyfile("../input/bert-part-one/bert_pytorch.bin", WORK_DIR + 'pytorch_model.bin')
shutil.copyfile( BERT_MODEL_PATH+ 'bert_config.json', WORK_DIR + 'bert_config.json')

In [ ]:
os.listdir("../working")

In [ ]:
# This is the Bert configuration file
from pytorch_pretrained_bert import BertConfig

bert_config = BertConfig(BERT_MODEL_PATH+'bert_config.json')


In [ ]:

def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

In [ ]:
#BERT_MODEL_PATH   = "../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/"
BERT_MODEL_PATH   = "../input/russian-bert-pavlov/"

In [ ]:
symbols_to_isolate = '.,?!-;*"…:—()%#$&_/@＼・ω+=”“[]^–>\\°<~•≠™ˈʊɒ∞§{}·τα❤☺ɡ|¢→̶`❥━┣┫┗Ｏ►★©―ɪ✔®\x96\x92●£♥➤´¹☕≈÷♡◐║▬′ɔː€۩۞†μ✒➥═☆ˌ◄½ʻπδηλσερνʃ✬ＳＵＰＥＲＩＴ☻±♍µº¾✓◾؟．⬅℅»Вав❣⋅¿¬♫ＣＭβ█▓▒░⇒⭐›¡₂₃❧▰▔◞▀▂▃▄▅▆▇↙γ̄″☹➡«φ⅓„✋：¥̲̅́∙‛◇✏▷❓❗¶˚˙）сиʿ✨。ɑ\x80◕！％¯−ﬂﬁ₁²ʌ¼⁴⁄₄⌠♭✘╪▶☭✭♪☔☠♂☃☎✈✌✰❆☙○‣⚓年∎ℒ▪▙☏⅛ｃａｓǀ℮¸ｗ‚∼‖ℳ❄←☼⋆ʒ⊂、⅔¨͡๏⚾⚽Φ×θ￦？（℃⏩☮⚠月✊❌⭕▸■⇌☐☑⚡☄ǫ╭∩╮，例＞ʕɐ̣Δ₀✞┈╱╲▏▕┃╰▊▋╯┳┊≥☒↑☝ɹ✅☛♩☞ＡＪＢ◔◡↓♀⬆̱ℏ\x91⠀ˤ╚↺⇤∏✾◦♬³の｜／∵∴√Ω¤☜▲↳▫‿⬇✧ｏｖｍ－２０８＇‰≤∕ˆ⚜☁'
symbols_to_delete = '\n🍕\r🐵😑\xa0\ue014\t\uf818\uf04a\xad😢🐶️\uf0e0😜😎👊\u200b\u200e😁عدويهصقأناخلىبمغر😍💖💵Е👎😀😂\u202a\u202c🔥😄🏻💥ᴍʏʀᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢ😋👏שלוםבי😱‼\x81エンジ故障\u2009🚌ᴵ͞🌟😊😳😧🙀😐😕\u200f👍😮😃😘אעכח💩💯⛽🚄🏼ஜ😖ᴠ🚲‐😟😈💪🙏🎯🌹😇💔😡\x7f👌ἐὶήιὲκἀίῃἴξ🙄Ｈ😠\ufeff\u2028😉😤⛺🙂\u3000تحكسة👮💙فزط😏🍾🎉😞\u2008🏾😅😭👻😥😔😓🏽🎆🍻🍽🎶🌺🤔😪\x08‑🐰🐇🐱🙆😨🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚地獄谷улкнПоАН🐾🐕😆ה🔗🚽歌舞伎🙈😴🏿🤗🇺🇸мυтѕ⤵🏆🎃😩\u200a🌠🐟💫💰💎эпрд\x95🖐🙅⛲🍰🤐👆🙌\u2002💛🙁👀🙊🙉\u2004ˢᵒʳʸᴼᴷᴺʷᵗʰᵉᵘ\x13🚬🤓\ue602😵άοόςέὸתמדףנרךצט😒͝🆕👅👥👄🔄🔤👉👤👶👲🔛🎓\uf0b7\uf04c\x9f\x10成都😣⏺😌🤑🌏😯ех😲Ἰᾶὁ💞🚓🔔📚🏀👐\u202d💤🍇\ue613小土豆🏡❔⁉\u202f👠》कर्मा🇹🇼🌸蔡英文🌞🎲レクサス😛外国人关系Сб💋💀🎄💜🤢َِьыгя不是\x9c\x9d🗑\u2005💃📣👿༼つ༽😰ḷЗз▱ц￼🤣卖温哥华议会下降你失去所有的钱加拿大坏税骗子🐝ツ🎅\x85🍺آإشء🎵🌎͟ἔ油别克🤡🤥😬🤧й\u2003🚀🤴ʲшчИОРФДЯМюж😝🖑ὐύύ特殊作戦群щ💨圆明园קℐ🏈😺🌍⏏ệ🍔🐮🍁🍆🍑🌮🌯🤦\u200d𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪天一家⚲\u2006⚭⚆⬭⬯⏖新✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴看ʁ𝗪𝗵𝗲𝗻𝘆𝗼𝘂𝗿𝗮𝗹𝗶𝘇𝗯𝘁𝗰𝘀𝘅𝗽𝘄𝗱📺ϖ\u2000үսᴦᎥһͺ\u2007հ\u2001ɩｙｅ൦ｌƽｈ𝐓𝐡𝐞𝐫𝐮𝐝𝐚𝐃𝐜𝐩𝐭𝐢𝐨𝐧Ƅᴨןᑯ໐ΤᏧ௦Іᴑ܁𝐬𝐰𝐲𝐛𝐦𝐯𝐑𝐙𝐣𝐇𝐂𝐘𝟎ԜТᗞ౦〔Ꭻ𝐳𝐔𝐱𝟔𝟓𝐅🐋ﬃ💘💓ё𝘥𝘯𝘶💐🌋🌄🌅𝙬𝙖𝙨𝙤𝙣𝙡𝙮𝙘𝙠𝙚𝙙𝙜𝙧𝙥𝙩𝙪𝙗𝙞𝙝𝙛👺🐷ℋ𝐀𝐥𝐪🚶𝙢Ἱ🤘ͦ💸ج패티Ｗ𝙇ᵻ👂👃ɜ🎫\uf0a7БУі🚢🚂ગુજરાતીῆ🏃𝓬𝓻𝓴𝓮𝓽𝓼☘﴾̯﴿₽\ue807𝑻𝒆𝒍𝒕𝒉𝒓𝒖𝒂𝒏𝒅𝒔𝒎𝒗𝒊👽😙\u200cЛ‒🎾👹⎌🏒⛸公寓养宠物吗🏄🐀🚑🤷操美𝒑𝒚𝒐𝑴🤙🐒欢迎来到阿拉斯ספ𝙫🐈𝒌𝙊𝙭𝙆𝙋𝙍𝘼𝙅ﷻ🦄巨收赢得白鬼愤怒要买额ẽ🚗🐳𝟏𝐟𝟖𝟑𝟕𝒄𝟗𝐠𝙄𝙃👇锟斤拷𝗢𝟳𝟱𝟬⦁マルハニチロ株式社⛷한국어ㄸㅓ니͜ʖ𝘿𝙔₵𝒩ℯ𝒾𝓁𝒶𝓉𝓇𝓊𝓃𝓈𝓅ℴ𝒻𝒽𝓀𝓌𝒸𝓎𝙏ζ𝙟𝘃𝗺𝟮𝟭𝟯𝟲👋🦊多伦🐽🎻🎹⛓🏹🍷🦆为和中友谊祝贺与其想象对法如直接问用自己猜本传教士没积唯认识基督徒曾经让相信耶稣复活死怪他但当们聊些政治题时候战胜因圣把全堂结婚孩恐惧且栗谓这样还♾🎸🤕🤒⛑🎁批判检讨🏝🦁🙋😶쥐스탱트뤼도석유가격인상이경제황을렵게만들지않록잘관리해야합다캐나에서대마초와화약금의품런성분갈때는반드시허된사용🔫👁凸ὰ💲🗯𝙈Ἄ𝒇𝒈𝒘𝒃𝑬𝑶𝕾𝖙𝖗𝖆𝖎𝖌𝖍𝖕𝖊𝖔𝖑𝖉𝖓𝖐𝖜𝖞𝖚𝖇𝕿𝖘𝖄𝖛𝖒𝖋𝖂𝕴𝖟𝖈𝕸👑🚿💡知彼百\uf005𝙀𝒛𝑲𝑳𝑾𝒋𝟒😦𝙒𝘾𝘽🏐𝘩𝘨ὼṑ𝑱𝑹𝑫𝑵𝑪🇰🇵👾ᓇᒧᔭᐃᐧᐦᑳᐨᓃᓂᑲᐸᑭᑎᓀᐣ🐄🎈🔨🐎🤞🐸💟🎰🌝🛳点击查版🍭𝑥𝑦𝑧ＮＧ👣\uf020っ🏉ф💭🎥Ξ🐴👨🤳🦍\x0b🍩𝑯𝒒😗𝟐🏂👳🍗🕉🐲چی𝑮𝗕𝗴🍒ꜥⲣⲏ🐑⏰鉄リ事件ї💊「」\uf203\uf09a\uf222\ue608\uf202\uf099\uf469\ue607\uf410\ue600燻製シ虚偽屁理屈Г𝑩𝑰𝒀𝑺🌤𝗳𝗜𝗙𝗦𝗧🍊ὺἈἡχῖΛ⤏🇳𝒙ψՁմեռայինրւդձ冬至ὀ𝒁🔹🤚🍎𝑷🐂💅𝘬𝘱𝘸𝘷𝘐𝘭𝘓𝘖𝘹𝘲𝘫کΒώ💢ΜΟΝΑΕ🇱♲𝝈↴💒⊘Ȼ🚴🖕🖤🥘📍👈➕🚫🎨🌑🐻𝐎𝐍𝐊𝑭🤖🎎😼🕷ｇｒｎｔｉｄｕｆｂｋ𝟰🇴🇭🇻🇲𝗞𝗭𝗘𝗤👼📉🍟🍦🌈🔭《🐊🐍\uf10aლڡ🐦\U0001f92f\U0001f92a🐡💳ἱ🙇𝗸𝗟𝗠𝗷🥜さようなら🔼'

In [ ]:
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer1 = TreebankWordTokenizer()

isolate_dict = {ord(c):f' {c} ' for c in symbols_to_isolate}
remove_dict = {ord(c):f'' for c in symbols_to_delete}


def handle_punctuation(x):
    x = x.translate(remove_dict)
    x = x.translate(isolate_dict)
    return x

def handle_contractions(x):
    x = tokenizer1.tokenize(x)
    return x

def fix_quote(x):
    x = [x_[1:] if x_.startswith("'") else x_ for x_ in x]
    x = ' '.join(x)
    return x

def preprocess(x):
    x = handle_punctuation(x)
    x = handle_contractions(x)
    x = fix_quote(x)
    return x

In [ ]:
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()

In [ ]:
%%time
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case= True)

In [ ]:
# num 
#d = 1804874

#k = d - valid_size 
#num_to_load= 1000000                         #Train size to match time limit#
#valid_size= 104874 
#/kaggle/input/datatinkoff/sr_train.csv
#/kaggle/input/datatinkoff/sr_test.csv

train_df = pd.read_csv(os.path.join(Data_dir,"sr_train.csv")).sample(d,random_state=SEED)
print(train_df.shape)
#print(train_df.target.mean())
#print(train_df[train_df.target > 0].count())
#test = train_df[k:]

#train = train_df[:k].sort_values(by=['target'], ascending=False)

#train_df = train[:num_to_load].sample(frac=1, random_state=42).append(test)


In [ ]:
train_df.loc[train_df.job_title.isnull(), 'job_title']  = 'Пусто'

train_df.loc[train_df.header.isnull(), 'header']  = 'Пусто'




In [ ]:
train_df.head()

In [ ]:
print('loaded %d records' % len(train_df))



# Make sure all comment_text values are strings
#train_df['comment_text'] = train_df['comment_text'].astype(str) 

#num_to_load= len(train_df[:-100000][train_df.target > 0])                     #Train size to match time limit
#valid_size= len(train_df[-100000:]) 


#train_df  = train_df[:-100000][train_df.target > 0].append(train_df[-100000:])
def merge_text(x):
    
    
    return "{0} . {1} . {2}".format(x['header'], x['text'], x['job_title']) 
train_df['text_1'] = train_df[['header', 'text', "job_title"]].apply(lambda x: merge_text(x), axis=1)
train_df['text_1'] = train_df['text_1'].progress_apply(lambda x:str(x))


sequences = convert_lines(train_df["text_1"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)
#train_df=train_df.fillna(0)



In [ ]:
train_df.head()

In [ ]:
#train_df.loc[train_df.dislike ==1, 'target']  =0
#train_df.loc[train_df.like ==1, 'target']  =1
#train_df.loc[train_df.view ==1, 'target']  =0.7
#train_df.loc[train_df.skip ==1, 'target']  =0.4

In [ ]:
train_df.head()

In [ ]:

# List all identities
identity_columns = [
    'like', 	'skip', 	'dislike', 	'view' 	]
y_columns= ['like', 	'skip', 	'dislike', 	'view'  ]

train_df = train_df.drop(['text_1', 'header', 'text', 'job_title'],axis=1)
# convert target to 0,1
#train_df['target']=(train_df['target']>=0.5).astype(float)

In [ ]:

X = sequences[:num_to_load]                
y = train_df[y_columns].values[:num_to_load]
X_val = sequences[num_to_load:]                
y_val = train_df[y_columns].values[num_to_load:]


In [ ]:
test_df=train_df.tail(valid_size).copy()
#train_df=train_df.head(num_to_load)

In [ ]:

#del train_df
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), torch.tensor(y,dtype=torch.float))



In [ ]:
del X
del y
del sequences

In [ ]:
gc.collect()

In [ ]:
class FocalLoss(nn.Module):
    #alpha=1
    #reduce= False
    def __init__(self, alpha=0.25, gamma=1, logits=True, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss
        
kl = FocalLoss()







In [ ]:


gc.collect()


output_model_file = "bert_pytorch.bin"

lr=2e-5
batch_size = 32
accumulation_steps=1
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

#model = BertForSequenceClassification.from_pretrained("../input/bert-part-one/",cache_dir=None,num_labels=len(y_columns))
model = BertForSequenceClassification.from_pretrained("../working/",cache_dir=None,num_labels=len(y_columns))

model.zero_grad()
model = model.to(device)




param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
train = train_dataset

num_train_optimization_steps = int(EPOCHS*len(train)/batch_size/accumulation_steps)

optimizer = OpenAIAdam(optimizer_grouped_parameters,
                     lr=lr,
                     warmup=0.05,
                     t_total=num_train_optimization_steps)




model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
model=model.train()

tq = tqdm_notebook(range(EPOCHS))
print(tq)
cnt = 1
for epoch in tq:
    cnt+=1
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    avg_loss = 0.
    avg_accuracy = 0.
    lossf=None
    tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
    
    
    #if cnt==2:
    #    checkpoint = torch.load('../input/weightbert/bert_pytorch.bin')
    ##    model.load_state_dict(checkpoint['model_state_dict'])
    #    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    #    epoch = checkpoint['epoch']
    #    loss = checkpoint['loss']
    
    for i,(x_batch, y_batch) in tk0:
        if i <= 200000:
        
            optimizer.zero_grad()
            
            y_pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
            #print(y_pred)
            loss =  F.binary_cross_entropy_with_logits(y_pred,y_batch.to(device))
            #loss = ls.forward(self, y_pred,y_batch.to(device))
            #loss = forward(y_pred,y_batch.to(device))
            #####loss = kl.forward( y_pred,y_batch.to(device))
            #loss  = F.mse_loss(y_pred,y_batch.to(device)) #81
            #loss  = F.soft_margin_loss(y_pred,y_batch.to(device)) 
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
            if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
                optimizer.step()                            # Now we can do an optimizer step
                optimizer.zero_grad()
            if lossf:
                #lossf = 0.95*lossf+0.05*loss.item()
                lossf = 0.98*lossf+0.02*loss.item()
            else:
                lossf = loss.item()
            tk0.set_postfix(loss = lossf)
            avg_loss += loss.item() / len(train_loader)
            avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
    tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)
    
    
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, "{0}".format(output_model_file))
    #break
    


#torch.save(model.state_dict(), output_model_file)
#torch.save({
 #           'epoch': epoch,
  #          'model_state_dict': model.state_dict(),
   #         'optimizer_state_dict': optimizer.state_dict(),
    #        'loss': loss
     #       }, "1_{0}".format(output_model_file))

#torch.save(model.state_dict(), output_model_file)


In [ ]:
output_model_file = 'bert_pytorch.bin'
model = BertForSequenceClassification(bert_config,num_labels=len(y_columns))
checkpoint = torch.load(output_model_file)
#model.load_state_dict(output_model_file)
#model.load_state_dict(torch.load(output_model_file ))
#model.load_state_dict(torch.load(output_model_file ))
#checkpoint = torch.load('1_bert_pytorch.bin')
model.load_state_dict(checkpoint['model_state_dict'])
#model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
for param in model.parameters():
    param.requires_grad=False
model.eval()
valid_preds = np.zeros((len(X_val) ,   len(y_columns)))
valid = torch.utils.data.TensorDataset(torch.tensor(X_val,dtype=torch.long))
valid_loader = torch.utils.data.DataLoader(valid, batch_size=32, shuffle=False)

tk0 = tqdm_notebook(valid_loader)
for i,(x_batch,)  in enumerate(tk0):
    #print(i)
    pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
    #print(pred[:,1].detach().cpu().squeeze().numpy().shape)
    valid_preds[i*32:(i+1)*32,0]=pred[:,0].detach().cpu().squeeze().numpy()
    valid_preds[i*32:(i+1)*32,1]=pred[:,1].detach().cpu().squeeze().numpy()
    valid_preds[i*32:(i+1)*32,2]=pred[:,2].detach().cpu().squeeze().numpy()
    valid_preds[i*32:(i+1)*32,3]=pred[:,3].detach().cpu().squeeze().numpy()




In [ ]:
print(os.listdir("../working/"))

In [ ]:
# From baseline kernel

def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]>0.5
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)



SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]>0.5]
    return compute_auc((subgroup_examples[label]>0.5), subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[(df[subgroup]>0.5) & (df[label]<=0.5)]
    non_subgroup_positive_examples = df[(df[subgroup]<=0.5) & (df[label]>0.5)]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[(df[subgroup]>0.5) & (df[label]>0.5)]
    non_subgroup_negative_examples = df[(df[subgroup]<=0.5) & (df[label]<=0.5)]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]>0.5])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)


In [ ]:
valid_preds.shape

In [ ]:
test_df

In [ ]:
print(valid_preds[:,0].shape)
print(test_df.shape)

In [ ]:

MODEL_NAME = ['model1', 'model2', 'model3', 'model4']
for i in range(len(MODEL_NAME)):
    test_df[MODEL_NAME[i]]=torch.sigmoid(torch.tensor(valid_preds[:,i])).numpy()
#TOXICITY_COLUMN = 'target'
test_df.head()


In [ ]:
sp = ['like', 'skip','dislike', 'view']
for i in range(len(MODEL_NAME)):
    TOXICITY_COLUMN = sp[i]
    bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME[i], sp[i])
    bias_metrics_df
    get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME[i]))

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
#like 	skip 	dislike 	view
y_true = np.array(test_df['like'])
y_scores = np.array(test_df['model1'])
roc_auc_score(y_true, y_scores)